# Frameworks d'Argumentation Étendus

**Navigation**: [← Tweety-6-Structured-Argumentation](Tweety-6-Structured-Argumentation.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-7b-Ranking-Probabilistic →](Tweety-7b-Ranking-Probabilistic.ipynb)

---

## Objectifs pédagogiques

1. Découvrir les Abstract Dialectical Frameworks (ADF) et leurs conditions d'acceptation
2. Maîtriser les frameworks bipolaires (Evidential, Necessity)
3. Explorer les frameworks pondérés (WAF) et sociaux (SAF)
4. Comprendre les Set Argumentation Frameworks (attaques collectives)
5. Appréhender les frameworks étendus avec attaques sur attaques (EAF/REAF)

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

> **Limitations connues (Tweety 1.28):**
> - ADF nécessite un solveur SAT incrémental natif (NativeMinisatSolver) - peut échouer avec fallback Sat4j
> - Les autres sections (Bipolar, WAF, SAF, SetAF, Extended) fonctionnent correctement

In [ ]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration des outils externes (standalone) ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# Auto-detection Clingo
clingo_paths = [
    shutil.which("clingo"),
    shutil.which("clingo.exe"),
    pathlib.Path("ext_tools/clingo/clingo.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/clingo/clingo.exe"),
    pathlib.Path("../ext_tools/clingo/clingo.exe"),
]
for cp in clingo_paths:
    if cp:
        if isinstance(cp, str):
            cp_path = pathlib.Path(cp)
            EXTERNAL_TOOLS["CLINGO"] = str(cp_path.parent.resolve())
            break
        elif cp.exists():
            EXTERNAL_TOOLS["CLINGO"] = str(cp.parent.resolve())
            break

# Auto-detection SPASS
spass_paths = [
    shutil.which("SPASS"),
    shutil.which("SPASS.exe"),
    pathlib.Path("ext_tools/spass/SPASS.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/spass/SPASS.exe"),
    pathlib.Path("../ext_tools/spass/SPASS.exe"),
]
for sp in spass_paths:
    if sp:
        if isinstance(sp, str):
            EXTERNAL_TOOLS["SPASS"] = sp
            break
        elif sp.exists():
            EXTERNAL_TOOLS["SPASS"] = str(sp.resolve())
            break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = pathlib.Path("jdk-17-portable")
    if not jdk_portable.exists():
        jdk_portable = pathlib.Path("Argument_Analysis/jdk-17-portable")
    
    if jdk_portable.exists():
        zulu_dirs = list(jdk_portable.glob("zulu*"))
        if zulu_dirs:
            java_home = zulu_dirs[0]
            os.environ["JAVA_HOME"] = str(java_home.resolve())
            print(f"JDK portable trouve: {java_home.name}")
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
        print("JVM non disponible.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

if jvm_ready:
    print("JVM prete pour Tweety.")
    tools = [t for t, p in EXTERNAL_TOOLS.items() if p]
    if tools:
        print(f"Outils externes: {', '.join(tools)}")

## Partie 5a : Frameworks d'Argumentation Étendus

Cette section explore des extensions complexes des cadres de Dung, permettant de modéliser des scénarios de raisonnement plus riches avec des conditions d'acceptation spécifiques, des relations de support, des poids, et des attaques collectives ou récursives.

### 5.1 Abstract Dialectical Frameworks (ADF)

Les ADF [Brewka et al., 2013] généralisent les cadres de Dung. Au lieu d'une simple relation d'attaque, chaque argument (ou "statement") se voit associer une **condition d'acceptation** ($\phi_s$), qui est une formule propositionnelle définie sur les arguments parents.

* **Structure :** Un ADF est un triplet $(S, L, C)$ où $S$ est l'ensemble des "statements", $L \subseteq S \times S$ sont les liens, et $C = \{\phi_s\}_{s \in S}$ est l'ensemble des conditions d'acceptation.
* **Sémantiques :** Généralisent celles de Dung (admissible, complète, fondée, préférée, stable, modèle/2-valué).
* **Tweety :** `AbstractDialecticalFramework`, `Argument`, `AcceptanceCondition`, `KppADFFormatParser`.

> **Note:** Le raisonnement ADF nécessite souvent des solveurs SAT natifs (`NativeMinisatSolver`) qui peuvent ne pas être disponibles via JPype.

In [ ]:
# --- 5.1 Abstract Dialectical Frameworks (ADF) ---
print("\n--- 5.1 Abstract Dialectical Frameworks (ADF) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple ADF...")
    print("NOTE IMPORTANTE : Le raisonnement ADF dans Tweety repose sur des")
    print("         solveurs SAT natifs (Minisat, Lingeling, Picosat).")

    adf_imports_ok = False
    native_solver_load_failed = False
    try:
        import jpype; from jpype.types import *; import pathlib
        from java.io import File; from java.util import Collection

        from org.tweetyproject.arg.adf.syntax.adf import AbstractDialecticalFramework
        from org.tweetyproject.arg.adf.syntax import Argument as AdfArgument
        from org.tweetyproject.arg.adf.syntax.acc import AcceptanceCondition
        from org.tweetyproject.arg.adf.io import KppADFFormatParser
        from org.tweetyproject.arg.adf.semantics.link import SatLinkStrategy, LinkStrategy
        from org.tweetyproject.arg.adf.semantics.interpretation import Interpretation
        from org.tweetyproject.arg.adf.reasoner import AdmissibleReasoner, CompleteReasoner, GroundReasoner
        from org.tweetyproject.logics.pl.sat import Sat4jSolver
        from org.tweetyproject.arg.adf.sat import IncrementalSatSolver

        print("Imports ADF de base reussis.")
        adf_imports_ok = True

        # Tenter de charger un solveur natif
        adf_solver = None
        try:
             from org.tweetyproject.arg.adf.sat.solver import NativeMinisatSolver
             adf_solver = NativeMinisatSolver()
             print("Utilisation tentee de NativeMinisatSolver.")
        except Exception as e_native_init:
             native_solver_load_failed = True
             print(f"Echec chargement NativeMinisatSolver (attendu sans java.library.path): {e_native_init}")
             print("   Utilisation du fallback Sat4jSolver...")
             adf_solver = Sat4jSolver()

        print("\nTentative de creation de SatLinkStrategy...")
        link_strategy = None
        try:
            IncrementalSatSolver_class = jpype.JClass("org.tweetyproject.arg.adf.sat.IncrementalSatSolver")
            if isinstance(adf_solver, IncrementalSatSolver_class):
                 print(f"   Le solveur ({adf_solver.getClass().getSimpleName()}) est un IncrementalSatSolver.")
                 link_strategy = SatLinkStrategy(adf_solver)
                 print("SatLinkStrategy creee avec succes.")
            else:
                 print(f"   Le solveur fallback ({adf_solver.getClass().getSimpleName()}) N'EST PAS un IncrementalSatSolver.")
                 print("   Impossible de creer SatLinkStrategy requis par KppADFFormatParser.")

        except jpype.JException as e_link_java:
             print(f"Erreur Java lors de la creation de SatLinkStrategy: {e_link_java.message()}")
        except Exception as e_link_py:
              print(f"Erreur Python lors de la creation de SatLinkStrategy: {e_link_py}")

        if link_strategy is None:
             print("\nCONCLUSION ADF : Impossible de proceder.")
             print("   Le raisonnement ADF necessite un solveur SAT incremental natif.")
             if native_solver_load_failed:
                 print("   Celui-ci n'a pas pu etre charge (UnsatisfiedLinkError probable).")

    except ImportError as e: print(f"Erreur d'import pour ADF : {e}. Verifiez le JAR 'arg.adf'.")
    except jpype.JException as e_java: print(f"Erreur Java generale: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()

### 5.2 Frameworks Bipolaires (Evidential, Necessity)

Les cadres d'argumentation bipolaires étendent les AAFs de Dung en introduisant une relation de **support** distincte de l'attaque:

* **Evidential AF**: Support ensembliste + arguments "prima facie" (acceptés par défaut)
* **Necessity AF**: Support ensembliste interprété comme une "nécessité" (tous les supportants requis)

In [ ]:
# --- 5.2 Frameworks Bipolaires ---
print("\n--- 5.2 Frameworks Bipolaires ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution des exemples Bipolaires...")
    bipolar_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        from java.util import HashSet, Collection, Set as JavaSet

        from org.tweetyproject.arg.dung.syntax import Argument
        from org.tweetyproject.arg.bipolar.syntax import (
             EvidentialArgumentationFramework, NecessityArgumentationFramework,
             BArgument, ArgumentSet, Attack, Support,
             BinaryAttack, BinarySupport, SetAttack, SetSupport
             )
        from org.tweetyproject.arg.bipolar.reasoner.evidential import (
            SelfSupportingReasoner, ConflictFreeReasoner as EvidentialConflictFreeReasoner,
            AdmissibleReasoner as EvidentialAdmissibleReasoner, GroundedReasoner as EvidentialGroundedReasoner,
            CompleteReasoner as EvidentialCompleteReasoner, PreferredReasoner as EvidentialPreferredReasoner,
            StableReasoner as EvidentialStableReasoner
            )
        from org.tweetyproject.arg.bipolar.reasoner.necessity import (
            AdmissibleReasoner as NecessityAdmissibleReasoner, GroundedReasoner as NecessityGroundedReasoner,
            CompleteReasoner as NecessityCompleteReasoner, PreferredReasoner as NecessityPreferredReasoner,
            StableReasoner as NecessityStableReasoner
            )

        print("Imports pour Frameworks Bipolaires reussis.")
        bipolar_imports_ok = True

        # --- Exemple Evidential Argumentation ---
        if bipolar_imports_ok:
            print("\n--- Exemple Evidential AF ---")
            et = EvidentialArgumentationFramework()
            args_ev = {name: BArgument(name) for name in "abcdef"}
            for arg in args_ev.values(): et.add(arg)

            attack_b_a = SetAttack(args_ev['b'], args_ev['a'])
            attack_b_c = SetAttack(args_ev['b'], args_ev['c'])
            attack_c_b = SetAttack(args_ev['c'], args_ev['b'])
            attack_c_d = SetAttack(args_ev['c'], args_ev['d'])
            attack_d_f = SetAttack(args_ev['d'], args_ev['f'])
            attack_f_f = SetAttack(args_ev['f'], args_ev['f'])
            support_d_e = SetSupport(args_ev['d'], args_ev['e'])

            print("   Ajout des attaques et supports (Evidential)...")
            attacks_et = et.getAttacks()
            supports_et = et.getSupports()
            attacks_et.add(attack_b_a); attacks_et.add(attack_b_c)
            attacks_et.add(attack_c_b); attacks_et.add(attack_c_d)
            attacks_et.add(attack_d_f); attacks_et.add(attack_f_f)
            supports_et.add(support_d_e)
            print("   Relations ajoutees.")

            et.addPrimaFacie(args_ev['b']); et.addPrimaFacie(args_ev['c'])
            et.addPrimaFacie(args_ev['d']); et.addPrimaFacie(args_ev['f'])

            print("\nFramework Evidential cree:")
            print(str(et.prettyPrint()))

            print("\nCalcul des extensions Evidential:")
            try:
                print(f" - Self-Supporting: {SelfSupportingReasoner().getModels(et)}")
                print(f" - Grounded: {EvidentialGroundedReasoner().getModels(et)}")
                print(f" - Preferred: {EvidentialPreferredReasoner().getModels(et)}")
            except Exception as e_ev_reason: print(f"Erreur raisonnement Evidential: {e_ev_reason}")

            # --- Exemple Necessity Argumentation ---
            print("\n\n--- Exemple Necessity AF ---")
            nt = NecessityArgumentationFramework()
            args_nec = {name: BArgument(name) for name in "abcde"}
            for arg in args_nec.values(): nt.add(arg)

            attack_b_a_n = BinaryAttack(args_nec['b'], args_nec['a'])
            attack_e_a_n = BinaryAttack(args_nec['e'], args_nec['a'])
            attack_c_d_n = BinaryAttack(args_nec['c'], args_nec['d'])
            support_a_c_n = BinarySupport(args_nec['a'], args_nec['c'])
            support_b_b_n = BinarySupport(args_nec['b'], args_nec['b'])
            
            supportants_s3 = ArgumentSet()
            supportants_s3.add(args_nec['b']); supportants_s3.add(args_nec['d'])
            supportes_s3 = HashSet(); supportes_s3.add(args_nec['e'])
            support_set_n = SetSupport(supportants_s3, JObject(supportes_s3, JavaSet))

            print("   Ajout des attaques et supports (Necessity)...")
            attacks_nt = nt.getAttacks(); supports_nt = nt.getSupports()
            attacks_nt.add(attack_b_a_n); attacks_nt.add(attack_e_a_n); attacks_nt.add(attack_c_d_n)
            supports_nt.add(support_a_c_n); supports_nt.add(support_b_b_n); supports_nt.add(support_set_n)
            print("   Relations ajoutees.")

            print("\nFramework Necessity cree:")
            print(str(nt.prettyPrint()))

            print("\nCalcul des extensions Necessity:")
            try:
                print(f" - Grounded: {NecessityGroundedReasoner().getModels(nt)}")
                print(f" - Preferred: {NecessityPreferredReasoner().getModels(nt)}")
            except Exception as e_nec_reason: print(f"Erreur raisonnement Necessity: {e_nec_reason}")

    except ImportError as e: print(f"Erreur d'import Bipolaires: {e}.")
    except jpype.JException as e_java: print(f"Erreur Java generale Bipolaires: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue Bipolaires: {e_gen}"); import traceback; traceback.print_exc()

### 5.3 Frameworks Pondérés (WAF)

Les Cadres d'Argumentation Pondérés (Weighted Argumentation Frameworks - WAF) associent un **poids** à chaque attaque, représentant sa force ou son coût. L'évaluation de l'acceptabilité prend en compte ces poids via des **seuils** ou des **agrégations**.

Tweety utilise la structure des **semi-anneaux** (`Semiring`) pour définir comment les poids sont interprétés:
- `WeightedSemiring`: Poids numériques sommés (coûts)
- `FuzzySemiring`: Degrés de vérité dans [0, 1]
- `ProbabilisticSemiring`: Probabilités

In [ ]:
# --- 5.3 Frameworks Pondérés (WAF) ---
print("\n--- 5.3 Frameworks Ponderes (WAF) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple WAF...")
    waf_imports_ok = False
    try:
        import jpype
        from jpype.types import *

        from org.tweetyproject.arg.weighted.syntax import WeightedArgumentationFramework
        from org.tweetyproject.arg.dung.syntax import Argument, Attack, DungTheory
        from org.tweetyproject.math.algebra import WeightedSemiring
        from org.tweetyproject.arg.weighted.reasoner import (
             SimpleWeightedConflictFreeReasoner, SimpleWeightedAdmissibleReasoner,
             SimpleWeightedCompleteReasoner, SimpleWeightedPreferredReasoner,
             SimpleWeightedStableReasoner, SimpleWeightedGroundedReasoner
        )
        from org.tweetyproject.arg.dung.reasoner import (
            SimpleAdmissibleReasoner, SimpleCompleteReasoner, SimpleGroundedReasoner
        )

        print("Imports WAF et dependances reussis.")
        waf_imports_ok = True

        if waf_imports_ok:
            semiring = WeightedSemiring()
            waf = WeightedArgumentationFramework(semiring)

            a = Argument("a"); b = Argument("b"); c = Argument("c"); d = Argument("d"); e_arg = Argument("e")
            waf.add(a); waf.add(b); waf.add(c); waf.add(d); waf.add(e_arg)

            # Attaques pondérées: add(Attack, Double weight)
            waf.add(Attack(a, b), 7.0)
            waf.add(Attack(c, b), 8.0)
            waf.add(Attack(d, c), 8.0)
            waf.add(Attack(c, d), 9.0)
            waf.add(Attack(d, e_arg), 5.0)
            waf.add(Attack(e_arg, e_arg), 6.0)

            print("\nFramework Pondere (WAF) cree:")
            print(str(waf))

            print("\n--- Raisonnement Pondere ---")

            # Admissible Pondéré vs Standard
            print("\n* Admissible Pondere (alpha, gamma) vs Standard:")
            try:
                adm_reasoner_std = SimpleAdmissibleReasoner()
                weighted_adm_reasoner = SimpleWeightedAdmissibleReasoner()
                dung_theory_std = DungTheory(waf)

                adm_sets_std = adm_reasoner_std.getModels(dung_theory_std)
                print(f"  - Standard    : ({adm_sets_std.size()}) {adm_sets_std}")

                adm_sets_0_0 = weighted_adm_reasoner.getModels(waf, 0.0, 0.0)
                print(f"  - Pondere (a=0, g=0) : ({adm_sets_0_0.size()}) {adm_sets_0_0}")
                adm_sets_15_0 = weighted_adm_reasoner.getModels(waf, 15.0, 0.0)
                print(f"  - Pondere (a=15, g=0): ({adm_sets_15_0.size()}) {adm_sets_15_0}")
            except Exception as e_adm: print(f"   Erreur Weighted Admissible: {e_adm}")

            # Grounded Pondéré vs Standard
            print("\n* Grounded Pondere (alpha, gamma) vs Standard:")
            try:
                gr_reasoner_std = SimpleGroundedReasoner()
                weighted_gr_reasoner = SimpleWeightedGroundedReasoner()
                gr_set_std = gr_reasoner_std.getModel(dung_theory_std)
                print(f"  - Standard    : {{{gr_set_std}}}")

                gr_set_0_0 = weighted_gr_reasoner.getModel(waf, 0.0, 0.0)
                print(f"  - Pondere (a=0, g=0) : {{{gr_set_0_0}}}")
            except Exception as e_gr: print(f"   Erreur Weighted Grounded: {e_gr}")

    except ImportError as e: print(f"Erreur d'import pour WAF : {e}.")
    except jpype.JException as e_java: print(f"Erreur Java generale WAF: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue WAF: {e_gen}"); import traceback; traceback.print_exc()

### 5.4 Frameworks Sociaux (SAF)

Les Cadres d'Argumentation Sociaux (SAF) [Leite, Martins, 2011] permettent d'associer des **votes** (positifs ou négatifs) aux arguments. Ces votes influencent la force ou l'acceptabilité finale des arguments.

* **Sémantique ISS** (Iterated Schema Semantics): Calcule un score pour chaque argument basé sur les scores de ses attaquants et sur les votes, jusqu'à convergence.

In [ ]:
# --- 5.4 Frameworks Sociaux (SAF) ---
print("\n--- 5.4 Frameworks Sociaux (SAF) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple SAF...")
    saf_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        try:
            from org.tweetyproject.arg.social.syntax import SocialAbstractArgumentationFramework
            from org.tweetyproject.arg.dung.syntax import Argument, Attack
            from org.tweetyproject.arg.social.reasoner import IssReasoner
            from org.tweetyproject.arg.social.semantics import SimpleProductSemantics
            print("Imports SAF directs reussis.")
            saf_imports_ok = True
        except ImportError:
            print("Imports directs SAF echoues. Tentative avec JClass...")
            SocialAbstractArgumentationFramework = jpype.JClass("org.tweetyproject.arg.social.syntax.SocialAbstractArgumentationFramework")
            Argument = jpype.JClass("org.tweetyproject.arg.dung.syntax.Argument")
            Attack = jpype.JClass("org.tweetyproject.arg.dung.syntax.Attack")
            IssReasoner = jpype.JClass("org.tweetyproject.arg.social.reasoner.IssReasoner")
            SimpleProductSemantics = jpype.JClass("org.tweetyproject.arg.social.semantics.SimpleProductSemantics")
            print("Imports SAF via JClass reussis.")
            saf_imports_ok = True

        from java.util import Collection

        if saf_imports_ok:
            saf = SocialAbstractArgumentationFramework()
            A = Argument("A"); B = Argument("B"); C = Argument("C"); D = Argument("D")
            saf.add(A); saf.add(B); saf.add(C); saf.add(D)

            attack_ab = Attack(A, B); attack_bc = Attack(B, C)
            attack_cb = Attack(C, B); attack_cd = Attack(C, D)

            try:
                attacks_collection = saf.getAttacks()
                attacks_collection.add(attack_ab); attacks_collection.add(attack_bc)
                attacks_collection.add(attack_cb); attacks_collection.add(attack_cd)
                print("   Attaques ajoutees avec succes.")

                # Votes
                saf.voteUp(A, 3); saf.voteDown(A, 1)  # A: +2 net
                saf.voteUp(B, 2)                       # B: +2 net
                saf.voteUp(C, 2); saf.voteDown(C, 5)  # C: -3 net
                saf.voteUp(D, 2); saf.voteDown(D, 1)  # D: +1 net

                print("\nFramework Social (SAF) cree:")
                print(str(saf))

                print("\nCalcul du modele avec ISS (Iterated Schema Semantics)...")
                try:
                    iss_reasoner = IssReasoner(SimpleProductSemantics(0.01), 0.001)
                    iss_model = iss_reasoner.getModel(saf)
                    print("\nModele ISS (scores d'acceptabilite):\n", str(iss_model))
                except jpype.JException as e_iss_java: print(f"Erreur Java lors du raisonnement ISS: {e_iss_java.message()}")
                except Exception as e_iss_py: print(f"Erreur Python lors du raisonnement ISS: {e_iss_py}")

            except Exception as e_add_attack:
                 print(f"Erreur lors de l'ajout d'attaques via getAttacks().add(): {e_add_attack}")

    except ImportError as e: print(f"Erreur d'import initiale pour SAF : {e}.")
    except jpype.JException as e_java: print(f"Erreur Java generale: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()

### 5.5 Set Argumentation Frameworks (SetAF)

Les Set Argumentation Frameworks [Nielsen, Parsons, 2006] généralisent les attaques: c'est un **ensemble** d'arguments qui attaque collectivement un argument.

* **Attaque d'ensemble**: $(X, a)$ où $X$ est un ensemble non vide d'arguments et $a$ est l'argument attaqué
* **Défense**: Un argument $a$ est défendu par $E$ si pour chaque $X$ qui attaque $a$, un sous-ensemble de $E$ attaque un élément de $X$

In [ ]:
# --- 5.5 Set Argumentation Frameworks (SetAF) ---
print("\n--- 5.5 Set Argumentation Frameworks (SetAF) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple SetAF...")
    setaf_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        from java.util import HashSet, Set as JavaSet

        from org.tweetyproject.arg.setaf.syntax import SetAf, SetAttack
        from org.tweetyproject.arg.dung.syntax import Argument
        from org.tweetyproject.arg.setaf.reasoners import SimpleGroundedSetAfReasoner, SimpleAdmissibleSetAfReasoner, SimplePreferredSetAfReasoner

        print("Imports SetAF et dependances reussis.")
        setaf_imports_ok = True

        if setaf_imports_ok:
            set_af = SetAf()
            a = Argument("a"); b = Argument("b"); c = Argument("c"); d_arg = Argument("d")
            set_af.add(a); set_af.add(b); set_af.add(c); set_af.add(d_arg)

            # Créer les ensembles attaquants
            attacker_set1 = HashSet()
            attacker_set1.add(b); attacker_set1.add(d_arg)

            attacker_set2 = HashSet()
            attacker_set2.add(c); attacker_set2.add(a)

            # Ajouter les attaques d'ensemble
            set_attack1 = SetAttack(JObject(attacker_set1, JavaSet), a)
            set_attack2 = SetAttack(JObject(attacker_set2, JavaSet), c)
            set_af.add(set_attack1); set_af.add(set_attack2)

            print("\nFramework SetAF cree:")
            print(str(set_af))
            print(f"   Arguments: {set_af.getNodes()}")
            print(f"   Attaques Set: {set_af.getAttacks()}")

            print("\n--- Raisonnement SetAF ---")
            try:
                gr_setaf_reasoner = SimpleGroundedSetAfReasoner()
                gr_setaf_extension = gr_setaf_reasoner.getModel(set_af)
                print(f"\n* Extension Fondee (Grounded): {gr_setaf_extension}")

                adm_setaf_reasoner = SimpleAdmissibleSetAfReasoner()
                adm_setaf_extensions = adm_setaf_reasoner.getModels(set_af)
                print(f"\n* Extensions Admissibles ({adm_setaf_extensions.size()}): {adm_setaf_extensions}")

                pref_setaf_reasoner = SimplePreferredSetAfReasoner()
                pref_setaf_extensions = pref_setaf_reasoner.getModels(set_af)
                print(f"\n* Extensions Preferees ({pref_setaf_extensions.size()}): {pref_setaf_extensions}")

            except jpype.JException as e_reason_java: print(f"Erreur Java lors du raisonnement SetAF: {e_reason_java.message()}")
            except Exception as e_reason_py: print(f"Erreur Python lors du raisonnement SetAF: {e_reason_py}")

    except ImportError as e: print(f"Erreur d'import pour SetAF : {e}. Verifiez le JAR 'arg.setaf'.")
    except jpype.JException as e_java: print(f"Erreur Java generale SetAF: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue SetAF: {e_gen}"); import traceback; traceback.print_exc()

### 5.6 Frameworks Étendus (Attaques sur Attaques)

Les Frameworks d'Argumentation Étendus [Modgil, 2009] permettent de modéliser des scénarios où une **attaque peut être attaquée** par un argument. Cela représente des concepts comme la préférence entre attaques.

* **EAF** (Extended AF): Arguments attaquent des attaques $(c, (a, b))$
* **REAF** (Recursive Extended AF): Généralise les EAF en permettant aux arguments d'attaquer des attaques sur des attaques (récursivement)

In [ ]:
# --- 5.6 Frameworks Étendus (EAF / REAF) ---
print("\n--- 5.6 Frameworks Etendus (EAF / REAF) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple EAF/REAF...")
    eaf_reaf_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import Argument, Attack
        from org.tweetyproject.arg.extended.syntax import ExtendedTheory, RecursiveExtendedTheory, ExtendedAttack
        from org.tweetyproject.arg.extended.reasoner import SimpleExtendedCompleteReasoner, SimpleRecursiveExtendedCompleteReasoner

        print("Imports EAF/REAF reussis.")
        eaf_reaf_imports_ok = True

        if eaf_reaf_imports_ok:
            print("\n--- Exemple Extended AF (EAF) ---")
            eaf_theory = ExtendedTheory()
            a_eaf = Argument("a_e"); b_eaf = Argument("b_e"); c_eaf = Argument("c_e")
            d_eaf = Argument("d_e"); e_eaf = Argument("e_e")
            eaf_theory.add(a_eaf); eaf_theory.add(b_eaf); eaf_theory.add(c_eaf)
            eaf_theory.add(d_eaf); eaf_theory.add(e_eaf)

            # Attaques standard
            print("   Ajout attaques standard EAF...")
            eaf_theory.addAttack(a_eaf, b_eaf); eaf_theory.addAttack(b_eaf, a_eaf)
            eaf_theory.addAttack(c_eaf, d_eaf); eaf_theory.addAttack(d_eaf, c_eaf)
            print("   Attaques standard ajoutees.")

            # Objets Attack cibles
            attack_ba_target = Attack(b_eaf, a_eaf)
            attack_ab_target = Attack(a_eaf, b_eaf)
            attack_cd_target = Attack(c_eaf, d_eaf)

            # Attaques étendues (Arg -> Attack)
            print("   Ajout attaques etendues EAF...")
            eaf_theory.addAttack(c_eaf, attack_ba_target)
            eaf_theory.addAttack(d_eaf, attack_ab_target)
            eaf_theory.addAttack(e_eaf, attack_cd_target)
            print("   Attaques etendues ajoutees.")

            print("\nFramework EAF cree:")
            try: print(str(eaf_theory.prettyPrint()))
            except: print(str(eaf_theory))

            print(f"\nCalcul des Extensions Completes (EAF)...")
            try:
                eaf_reasoner = SimpleExtendedCompleteReasoner()
                eaf_extensions = eaf_reasoner.getModels(eaf_theory)
                print(f"Extensions Completes (EAF): ({eaf_extensions.size()})")
                for ext in eaf_extensions: print(f"  - {ext}")
            except Exception as e_eaf_reason: print(f"Erreur raisonnement EAF: {e_eaf_reason}")

            # --- Exemple REAF ---
            print("\n\n--- Exemple Recursive Extended AF (REAF) ---")
            reaf_theory = RecursiveExtendedTheory()
            a_reaf = Argument("a_r"); b_reaf = Argument("b_r"); c_reaf = Argument("c_r")
            d_reaf = Argument("d_r"); e_reaf = Argument("e_r"); f_reaf = Argument("f_r")
            reaf_theory.add(a_reaf); reaf_theory.add(b_reaf); reaf_theory.add(c_reaf)
            reaf_theory.add(d_reaf); reaf_theory.add(e_reaf); reaf_theory.add(f_reaf)

            print("   Ajout attaques standard REAF...")
            reaf_theory.addAttack(a_reaf, b_reaf); reaf_theory.addAttack(b_reaf, a_reaf)
            reaf_theory.addAttack(d_reaf, c_reaf); reaf_theory.addAttack(c_reaf, d_reaf)
            print("   Attaques standard ajoutees.")

            # ExtendedAttack cibles niveau 1
            attack_ab_reaf_target = ExtendedAttack(a_reaf, b_reaf)
            attack_ba_reaf_target = ExtendedAttack(b_reaf, a_reaf)
            attack_cd_reaf_target = ExtendedAttack(c_reaf, d_reaf)

            print("   Ajout attaques etendues REAF (niveau 1)...")
            reaf_theory.addAttack(c_reaf, JObject(attack_ba_reaf_target, ExtendedAttack))
            reaf_theory.addAttack(d_reaf, JObject(attack_ab_reaf_target, ExtendedAttack))
            reaf_theory.addAttack(e_reaf, JObject(attack_cd_reaf_target, ExtendedAttack))
            print("   Attaques etendues (niveau 1) ajoutees.")

            # Niveau 2
            attack_e_cd_target_lvl2 = ExtendedAttack(e_reaf, JObject(attack_cd_reaf_target, ExtendedAttack))
            print("   Ajout attaque etendue REAF (niveau 2)...")
            reaf_theory.addAttack(f_reaf, JObject(attack_e_cd_target_lvl2, ExtendedAttack))
            print("   Attaque etendue (niveau 2) ajoutee.")

            print("\nFramework REAF cree:")
            try: print(str(reaf_theory.prettyPrint()))
            except: print(str(reaf_theory))

            print(f"\nCalcul des Extensions Completes (REAF)...")
            try:
                reaf_reasoner = SimpleRecursiveExtendedCompleteReasoner()
                reaf_extensions = reaf_reasoner.getModels(reaf_theory)
                print(f"Extensions Completes (REAF): ({reaf_extensions.size()})")
                for ext in reaf_extensions: print(f"  - {ext}")
            except Exception as e_reaf_reason: print(f"Erreur raisonnement REAF: {e_reaf_reason}")

    except ImportError as e: print(f"Erreur d'import pour EAF/REAF : {e}. Verifiez le JAR 'arg.extended'.")
    except jpype.JException as e_java: print(f"Erreur Java generale EAF/REAF: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue EAF/REAF: {e_gen}"); import traceback; traceback.print_exc()

---

## Résumé

Ce notebook a couvert les frameworks d'argumentation étendus:
- **ADF**: Conditions d'acceptation spécifiques (limitation solveur SAT)
- **Bipolaires**: Evidential et Necessity (support + attaque)
- **WAF**: Frameworks pondérés avec seuils
- **SAF**: Frameworks sociaux avec votes et ISS
- **SetAF**: Attaques collectives (ensemble -> argument)
- **EAF/REAF**: Attaques sur attaques (récursif)

## Prochaines étapes

Le notebook suivant explore les sémantiques de classement (ranking) et l'argumentation probabiliste.

---

**Navigation**: [← Tweety-6-Structured-Argumentation](Tweety-6-Structured-Argumentation.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-7b-Ranking-Probabilistic →](Tweety-7b-Ranking-Probabilistic.ipynb)